- - - 
CASE A 

Generation of a new model from scratch, including database and data input

In [1]:
model_dir_name = 'model2'
main_dir_path = 'D:\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies'

In [2]:
# create model directory with essential setup files

from src.frontend import esm

esm.create_model_dir(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
    force_overwrite=False,
)

WARNING | default_logger.file_manager | Directory 'model2' already exists.
INFO | default_logger.file_manager | Folder and blank settings files for model 'model2' ready.


In [3]:
# generate model instance based on setup files filled by the user.
# first validates model directory (in case user relies on existing directory). 
# then parse settings and paths. 
# if specified in settings, generates blank sets file.

from src.frontend import esm

model = esm.Model(
    model_dir_name=model_dir_name,
    main_dir_path=main_dir_path,
)



INFO | Model | 'Model' object initialization...
DEBUG | Model.file_manager | 'FileManager' object generated.
INFO | Model | 'Model' object initialized.


In [2]:
import sys
print(sys.path)

['d:\\git_repos\\pyESM', 'C:\\ProgramData\\Anaconda3\\python.exe', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\python310.zip', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\DLLs', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\esm', '', 'C:\\Users\\matte\\AppData\\Roaming\\Python\\Python310\\site-packages', 'd:\\git_repos\\pyesm', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\lib\\site-packages', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\lib\\site-packages\\win32', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\esm\\lib\\site-packages\\Pythonwin']


- - - 
CASE B

Generation of a new model working with existing database and data input files.

(new_model_dir=False)

In [ ]:
# Generating a Model instance referring to existing model folder and input data.
from src.backend.model import Model

model = Model(
    model_dir_name='test_case_2',
    main_dir_path='D:\Politecnico di Milano\Documentale DENG - PRIN-MIMO\Models\pyESM_docs\case_studies',
    new_model_dir=False,
)

# test.load_model_coordinates()
# test.initialize_problem()

In [ ]:
# load model sets from the xlsx files filled by the user
# Note: data validation in excel according to the constant file is envisaged
test.load_model_coordinates()

In [ ]:
# Once sets has compiled in excel and loaded in the database, 
# blank database is generated.
# Note: an error is raised at the first launch of the method, 
# disappearing the second run.
test.generate_blank_database()

In [ ]:
# Load input file/s data into the sql database.
# Can be used to update data in the sql database without generating a new 
# Model instance
test.load_data_files_to_database(operation='overwrite')

In [ ]:
# Initialize cvxpy problem.and
# Fetch data from sql database to cvxpy exogenous variables,
# (to be done: generate equations and objective function)
test.initialize_problem()

In [ ]:
import yaml

yaml_data = """
v:
    symbol: 'v'
    name: 'make coefficients matrix'
    type: 'exogenous'
    coordinates: null
    shape: 
        technologies: {set_categories: Supply technology}
        flows: {set_categories: Product flow}

u:
    symbol: 'u'
    name: 'use coefficients matrix'
    type: 'exogenous'
    coordinates: null
    shape: 
        flows: {set_categories: Product flow}
        technologies: 
            set_categories: Product flow
            aggregation_key: competition
"""

data_dict = yaml.safe_load(yaml_data)

data_dict['u']['shape']


In [ ]:
# In case it is needed, update data without re-generate a new instance of Model class.
test.update_database_and_problem()

In [ ]:
test.load_results_to_database(operation='update')

- - - 
TESTS

In [ ]:
test.core.index.sets.technologies

In [ ]:
test.core.index.variables.X

In [ ]:
import numpy as np
values = np.ones((3,1))

variable.data['variable'][0].value = values

In [ ]:
import pandas as pd
from src.util import constants
from src.util import util

field_id = constants._STD_ID_FIELD['id'][0]

col_to_update = constants._STD_VALUES_FIELD['values'][0]
cols_common = list(variable.coordinates.keys())

new_df = variable.reshaping_variable_data(0)

new_values = {
    's_Name': 'Reference',
    'dt_Name': '2023',
    't_Name': 'Final demand',
    'values': 15.0
}

new_df.loc[len(new_df)] = new_values
new_df.at[0,'values'] = 2

test.core.sqltools.open_connection()
existing_df = test.core.sqltools.table_to_dataframe(table_name=variable.symbol)
existing_df.drop(columns=['id'],inplace=True)
test.core.sqltools.close_connection()

print('\n')
print('existing_df: \n', existing_df)
print('\n')
print('new_df: \n', new_df)

In [ ]:
variable=test.core.index.variables.u

test.core.database.sqltools.open_connection()
raw_data = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name=variable.symbol,
    filters_dict=variable.data['filter'][0]
)
test.core.database.sqltools.close_connection()

raw_data

In [ ]:
test.core.database.sqltools.open_connection()

var = test.core.database.sqltools.filtered_table_to_dataframe(
    table_name='u',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        'f_Name': ['Energy', 'Steel'],
        't_Name': ['PV power plant', 'Steel factory', 'Gas power plant']
    }
)

# filter_1 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='f_Name',
#     parent_table_name='_set_FLOWS',
#     parent_table_fields={
#         'f_Category': ['Product flow'],
#     }
# )

# filter_2 = test.core.database.sqltools.get_related_table_keys(
#     child_column_name='dt_Name',
#     parent_table_name='_set_DATETIME',
#     parent_table_fields={
#         'dt_Name': [2023],
#     }
# )

# var_custom_filter = test.core.database.sqltools.filtered_table_to_dataframe(
#     table_name='v',
#     filters_dict={
#         **filter_1,
#         **filter_2,
#         **{'f_Name': ['Steel']},
#     }
# )

test.core.database.sqltools.close_connection()

# filter_1, filter_2
var

Other commands

In [ ]:
# delete all the information included in the model data folder, if necessary
test.erase_model()